In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate, AUROC

In [2]:
bs = 16

In [3]:
np.random.seed(233)
data = ImageDataBunch.from_folder(path='/home/ike/Data/kaggledayschina/kaggledays-china/train3c', train='.', valid_pct=0.2, ds_tfms=get_transforms(), size=224, bs=bs).normalize(imagenet_stats)

In [4]:
data.classes

['nonstar', 'star']

In [5]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate, AUROC()])

In [6]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate,auroc,time
0,0.618307,0.576889,0.222112,0.529246,01:03
1,0.500455,0.432500,0.166335,0.686045,01:01


In [15]:
learn.export('/home/ike/Data/kaggledayschina/kaggledays-china/export.pkl')

In [10]:
import pandas as pd
import numpy as np

submission = pd.read_csv("/home/ike/Data/kaggledayschina/kaggledays-china/test.csv")

In [11]:
submission['fileName'] = submission.id + '.png'

In [12]:
testset = ImageList.from_df(submission, path="/home/ike/Data/kaggledayschina/kaggledays-china/test3c/test3c_data/", cols = 'fileName')

In [13]:
len(testset)

1257

In [16]:
learn = load_learner('/home/ike/Data/kaggledayschina/kaggledays-china/', test=testset)
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [17]:
preds

tensor([[0.8361, 0.1639],
        [0.9194, 0.0806],
        [0.9237, 0.0763],
        ...,
        [0.9238, 0.0762],
        [0.9838, 0.0162],
        [0.9200, 0.0800]])

In [20]:
submission['is_star'] = np.array(preds.tolist())[:, 1]

In [22]:
submission[['id', 'is_star']].to_csv("submission_01.csv", header=True, index=False)